In [ ]:
from Network_calculation import Network_cal
from CNN_generator import Epoch, Encoder, Decoder, EEGDataset
from Save_BO import SaveData
import matplotlib.pyplot as plt # plotting library
import numpy as np # this module is useful to work with numerical arrays
import pandas as pd 
import random 
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader,random_split, Dataset
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import math
import GPyOpt


In [ ]:
Savedata = SaveData()
encoder = Encoder()
decoder = Decoder()
network_calc = Network_cal()
train = Epoch()

In [ ]:
train_data = np.load("train_data_1_30.npy")
train_labels = np.load("train_label_1_30.npy").astype(np.int32)
test_data = np.load("test_data_31_40.npy")
test_labels = np.load("test_label_31_40.npy").astype(np.int32)

batch_size=32

#train_data = np.transpose(train_data, (1,2,0))

train_data = train_data*1e5
test_data = test_data*1e5
train_dataset = EEGDataset(train_data, train_labels)
test_dataset = EEGDataset(test_data, test_labels)


train_transform = transforms.Compose([
transforms.ToTensor(),
#transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

test_transform = transforms.Compose([
transforms.ToTensor(),
#transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_dataset.transform = train_transform
test_dataset.transform = test_transform

m=len(train_dataset)

#print(test_dataset[:,1])


print(m)
train_data, val_data = random_split(train_dataset, [math.floor(m*0.8), math.ceil(m*0.2)])
#print(train_data[:5])
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,shuffle=True)
valid_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,shuffle=True)

In [ ]:
E = encoder(encoder_list, linear_int,in_channels = in_channels, out_channels = out_channels, encoded_space_dim = 35, fc2_input_dim = 35, layers=3, filter_size =32, kernel = (1,4), kernel_p = (1,2), stride = 2, stride_p = 2, padding = (0,1), padding_p = (0,0), pooling = True)
D = decoder(decoder_list, linear_int,in_channels = in_channels, out_channels = out_channels, encoded_space_dim = 35, fc2_input_dim = 35, layers=3, filter_size =32, kernel = (1,4), kernel_p = (1,2), stride = 2, stride_p = 2, padding = (0,1), padding_p = (0,0), pooling = True)


In [ ]:
#Domain settings for architecture.

Domain_archi = [{'name': 'in_channels', 'type': 'discrete', 'domain': encoder.in_channels}, #These domains are lists - Hence it will be lists in a list
          {'name': 'out_channels', 'type': 'discrete', 'domain': encoder.out_channels}, #These domains are lists - Hence it will be lists in a list
          {'name': 'encoded_space_dim', 'type': 'discrete', 'domain': encoder.encoded_space_dim},
          {'name': 'fc2_input_dim', 'type': 'discrete', 'domain': encoder.fc2_input_dim},
          {'name': 'layers', 'type': 'discrete', 'domain': encoder.layers},
          {'name': 'kernel', 'type': 'discrete', 'domain': encoder.kernel},
          {'name': 'kernel_p', 'type': 'discrete', 'domain': encoder.kernel_p},
          {'name': 'stride', 'type': 'discrete', 'domain': encoder.stride},
          {'name': 'stride_p', 'type': 'discrete', 'domain': encoder.stride_p},
          {'name': 'padding', 'type': 'discrete', 'domain': encoder.padding},
          {'name': 'padding_p', 'type': 'discrete', 'domain': encoder.padding_p},
          {'name': 'pooling', 'type': 'discrete', 'domain': encoder.pooling}
         ]

In [ ]:
#Domain settings for hyperparameters

Domain_hyper = [{'name': 'in_channels', 'type': 'discrete', 'domain': encoder.in_channels}, #These domains are lists - Hence it will be lists in a list
          {'name': 'out_channels', 'type': 'discrete', 'domain': encoder.out_channels}, #These domains are lists - Hence it will be lists in a list
          {'name': 'encoded_space_dim', 'type': 'discrete', 'domain': encoder.encoded_space_dim},
          {'name': 'fc2_input_dim', 'type': 'discrete', 'domain': encoder.fc2_input_dim},
          {'name': 'layers', 'type': 'discrete', 'domain': encoder.layers},
          {'name': 'kernel', 'type': 'discrete', 'domain': encoder.kernel},
          {'name': 'kernel_p', 'type': 'discrete', 'domain': encoder.kernel_p},
          {'name': 'stride', 'type': 'discrete', 'domain': encoder.stride},
          {'name': 'stride_p', 'type': 'discrete', 'domain': encoder.stride_p},
          {'name': 'padding', 'type': 'discrete', 'domain': encoder.padding},
          {'name': 'padding_p', 'type': 'discrete', 'domain': encoder.padding_p},
          {'name': 'pooling', 'type': 'discrete', 'domain': encoder.pooling}
         ]

In [ ]:
class Bayersian_optimization():
                                            #Domain: [{'name': 'batch_size', 'type': 'discrete', 'domain': batch_size},{}]
                                            #param_list = [batch_size = tuple(np.arange(64,128,1)), learning_rate = tuple(np.arange(1,31,0.5)*1e-5)]


    def __init__(self, domain=None,param_list=[],model=None,device_type='cuda'):
        self.domain = domain
        
        self.param_list = param_list
        self.model = model
        self.device_type = device_type
        super().__init__()                           
    
    def objective_function(self,x):

        #print(x)
        # we have to handle the categorical variables that is convert 0/1 to labels
        # log2/sqrt and gini/entropy
        param = x[0]
        #print(int(param[0]),param[1])
        if param[11] == True:
            pooling = True
        else:
            pooling = False
        # we have to handle the categorical variables
        #if param[2] == 0:
        #    max_f = 'log2'
        #elif param[2] == 1:
        # max_f = 'sqrt'
        #else:
        #  max_f = None

        #if param[3] == 0:
        #  crit = 'gini'
        #else:
        #crit = 'entropy'

        #create the model
        model = self.model(encoder, decoder, device)
        device = torch.device(self.device_type)  # use cuda or cpu
        model.to(device)
        model2 = train(encoder, decoder, device) #dataloader, loss_fn, optimizer,n=10))
        
        # fit the model 
        
        test_accuracy = model2.training()

        #model.fit(Xtrain, ytrain)
        print(test_accuracy)
        return - test_accuracy

    def bo(self,max_iter=15,exloration_weight=0.05,acquisition='EI'):
        opt = GPyOpt.methods.BayesianOptimization(f = objective_function,   # function to optimize
                                                    domain = self.domain,         # box-constrains of the problem
                                                    acquisition_type = acquisition,      # Select acquisition function MPI, EI, LCB
                                                    )
        opt.acquisition.exploration_weight=exloration_weight

        opt.run_optimization(max_iter = max_iter) 

        x_best = opt.X[np.argmin(opt.Y)]
        print("The best parameters obtained: batch_size=" + str(x_best[0]) + ", learning_rate=" + str(x_best[1]))
        return x_best, architecture, hyperparameters